In [1]:

import kaldi_features
import scipy.io.wavfile

In [2]:
class KaldiFeaturesExtractor:

    def __init__(self, params: dict):
        self.__caldi_features_calcer = kaldi_features.KaldiFeaturesCalcer(params)

    def extract(self, path):
        wave = scipy.io.wavfile.read(path)
        return self.__caldi_features_calcer.compute(wave[1].astype(np.float16))

    @classmethod
    def create(cls, config):
        return KaldiFeaturesExtractor(config)

In [3]:
import json

In [4]:
with open("features_config.json") as f:
    feature_calcer_params = json.loads(f.read())

In [5]:
feature_calcer_params

{'sample-frequency': 16000.0,
 'frame-length': 20.0,
 'frame-shift': 10.0,
 'preemphasis-coefficient': 0.97,
 'remove-dc-offset': True,
 'dither': 0.01,
 'window-type': 'hamming',
 'blackman-coeff': 0.42,
 'round-to-power-of-two': True,
 'snip-edges': True,
 'num-mel-bins': 80,
 'use-mel-scale': True,
 'low-freq': 0.0,
 'high-freq': 0.0,
 'vtln-low': 100.0,
 'vtln-high': -500.0,
 'debug-mel': False,
 'use-energy': False,
 'energy-floor': 0.0,
 'raw-energy': True,
 'htk-compat': False,
 'use-log-fbank': True,
 'use-power': True}

In [6]:
KFE = KaldiFeaturesExtractor(feature_calcer_params)

In [7]:
import io
import numpy as np

In [25]:
for i in range(1, 75):
    res = KFE.extract("wavs/Post_Russia_Recordings_wav/"+str(i)+".wav")
    np.savetxt('csvs/' + str(i) + ".csv", res, delimiter=',')

TypeError: only size-1 arrays can be converted to Python scalars

In [9]:
result

array([[-11.773279  , -12.108532  , -13.9570465 , ...,  -0.94368804,
         -0.4717266 ,  -1.2321846 ],
       [ -2.9196281 ,  -2.3802102 ,  -1.789802  , ...,   5.044058  ,
          5.2834787 ,   4.579864  ],
       [  0.98746306,   1.0353358 ,   1.0905256 , ...,   5.782383  ,
          5.965891  ,   5.6404514 ],
       ...,
       [  0.5769106 ,   2.8423622 ,   3.8623774 , ...,  10.806189  ,
          9.904436  ,   8.908124  ],
       [ -0.30312175,   0.6563626 ,   1.4488351 , ...,   9.607957  ,
          6.6755686 ,   5.200448  ],
       [  1.308285  ,   2.223018  ,   2.999321  , ...,   7.3784375 ,
          5.8190026 ,   4.779843  ]], dtype=float32)

In [10]:
np.savetxt('1wav.csv', result, delimiter=',')

In [11]:
result.shape

(514, 80)

In [12]:
 class Loader:
        def __init__(self, path):
            self.load_graph(path)
            
        def load_graph(self, model_filepath):
            print('Loading model...')
            self.graph = tf.Graph()
            self.sess = tf.InteractiveSession(graph = self.graph, config=tf.ConfigProto(allow_soft_placement=True))

            with tf.gfile.GFile(model_filepath, 'rb') as f:
                graph_def = tf.GraphDef()
                graph_def.ParseFromString(f.read())

            print('Check out the input placeholders:')
            nodes = [n.name + ' => ' +  n.op for n in graph_def.node if n.op in ('Placeholder')]
            #nodes = [n.name + ' => ' +  n.op for n in graph_def.node]
            for node in nodes:
                print(node)

            self.input_features = tf.placeholder(tf.float16, shape = [1, 514, 80], name='input_features')
            self.input_lengths = tf.placeholder(tf.int32, shape = [1], name='input_lengths')

            tf.import_graph_def(graph_def, {'input_features': self.input_features, 'input_lengths': self.input_lengths})

            print('Model loading complete!')
            
        def test(self, data):

            output_tensor = self.graph.get_tensor_by_name("")
            output = self.sess.run(output_tensor, feed_dict = {self.input_features: data, self.input_lengths: np.array([[514]])})

            return output
            

In [13]:
data = result.reshape((1, 80, 514))

In [13]:
import tensorflow as tf

/home/admin/anaconda3/envs/tf1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/admin/anaconda3/envs/tf1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/admin/anaconda3/envs/tf1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/admin/anaconda3/envs/tf1/lib/python3.6/site-packages

In [14]:
loader = Loader('saved_model.pb')

Loading model...
Check out the input placeholders:
input_features => Placeholder
input_lengths => Placeholder


ValueError: NodeDef missing attr 'dtype' from Op<name=Const; signature= -> output:dtype; attr=value:tensor; attr=dtype:type>; NodeDef: {{node import/ForwardPass/w2l_encoder/conv121/kernel}}

In [ ]:
class TFWrapper(object):

    def __init__(self, model_filepath):
        # The file path of model
        self.model_filepath = model_filepath
        # Initialize the model
        self.load_graph(model_filepath = self.model_filepath)

    def load_graph(self, model_filepath):
        '''
        Lode trained model.
        '''
        print('Loading model...')
        self.graph = tf.Graph()
        self.sess = tf.InteractiveSession(graph = self.graph)

        with tf.gfile.GFile(model_filepath, 'rb') as f:
            graph_def = tf.GraphDef()
            graph_def.ParseFromString(f.read())

        print('Check out the input placeholders:')
        nodes = [n.name + ' => ' +  n.op for n in graph_def.node if n.op in ('Placeholder')]
        for node in nodes:
            print(node)

        # Define input tensor
        feature_count = 80
        num_frames = None
        self.input = tf.placeholder(np.float32, shape = [None, feature_count, num_frames], name='features')
        tf.import_graph_def(graph_def, {'features': self.input})

        print('Model loading complete!')
       

    def apply(self, features):

        # Know your output node name
        output_tensor = self.graph.get_tensor_by_name("import/logits:0")
        output = self.sess.run(output_tensor, feed_dict = {self.input: features})

        return output